In [1]:
!pip install nltk
!pip install openpyxl

In [2]:
import pandas as pd
import re
import nltk
from nltk.util import ngrams
from collections import Counter
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/codespace/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
df=pd.read_excel('/workspaces/BMIG60303-assignments/All_Articles_Excel_Dec2019July2020.xlsx')

In [4]:
df.head()

,Date Added,Author,Title,Abstract,Year,Journal/Publisher,Volume,Issue,Pages,Accession Number,...,Unnamed: 84,Unnamed: 85,Unnamed: 86,Unnamed: 87,Unnamed: 88,Unnamed: 89,Unnamed: 90,Unnamed: 91,Unnamed: 92,Unnamed: 93
0,2020-07-31,"Zwerling, A",Understanding spending trends for tuberculosis,Total out-of-pocket spending decreased over th...,2020,Lancet Infectious Diseases,20,8,879-880,2428388309,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2020-07-31,"Zucoloto, GM, Pedro; Porto, Patricia",A propriedade industrial pode limitar o combat...,Esta nota técnica apresenta alguns dos pontos ...,2020,NaN,NaN,Issue,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2020-07-31,"Zimmer, MAZ, Anne K.; Weißer, Christian W.; Vo...",Hypernatremia—A Manifestation of COVID-19: A C...,We report for the first time therapy-resistant...,2020,A&A Practice,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2020-07-31,"Zignale, M",The experienced space between mobility and Cov...,If it is true that lived space represents our ...,2020,Documenti Geografici,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2020-07-31,"Zhu, XC, Bodong; Avadhanam, Rukmini Manasa; Sh...",Reading and connecting: using social annotatio...,Purpose - The COM-19 pandemic has forced many ...,2020,Information and Learning Sciences,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
df.columns

Index(['Date Added', 'Author', 'Title', 'Abstract', 'Year',
       'Journal/Publisher', 'Volume', 'Issue', 'Pages', 'Accession Number',
       'DOI', 'URL', 'Name of Database', 'Database Provider', 'Language',
       'Keywords', 'Unnamed: 16', 'Unnamed: 17', 'Unnamed: 18', 'Unnamed: 19',
       'Unnamed: 20', 'Unnamed: 21', 'Unnamed: 22', 'Unnamed: 23',
       'Unnamed: 24', 'Unnamed: 25', 'Unnamed: 26', 'Unnamed: 27',
       'Unnamed: 28', 'Unnamed: 29', 'Unnamed: 30', 'Unnamed: 31',
       'Unnamed: 32', 'Unnamed: 33', 'Unnamed: 34', 'Unnamed: 35',
       'Unnamed: 36', 'Unnamed: 37', 'Unnamed: 38', 'Unnamed: 39',
       'Unnamed: 40', 'Unnamed: 41', 'Unnamed: 42', 'Unnamed: 43',
       'Unnamed: 44', 'Unnamed: 45', 'Unnamed: 46', 'Unnamed: 47',
       'Unnamed: 48', 'Unnamed: 49', 'Unnamed: 50', 'Unnamed: 51',
       'Unnamed: 52', 'Unnamed: 53', 'Unnamed: 54', 'Unnamed: 55',
       'Unnamed: 56', 'Unnamed: 57', 'Unnamed: 58', 'Unnamed: 59',
       'Unnamed: 60', 'Unnamed: 61', 'Unn

In [6]:
df = df[['Date Added', 'Author', 'Title', 'Abstract', 'Year']]

In [7]:
df.head()

,Date Added,Author,Title,Abstract,Year
0,2020-07-31,"Zwerling, A",Understanding spending trends for tuberculosis,Total out-of-pocket spending decreased over th...,2020
1,2020-07-31,"Zucoloto, GM, Pedro; Porto, Patricia",A propriedade industrial pode limitar o combat...,Esta nota técnica apresenta alguns dos pontos ...,2020
2,2020-07-31,"Zimmer, MAZ, Anne K.; Weißer, Christian W.; Vo...",Hypernatremia—A Manifestation of COVID-19: A C...,We report for the first time therapy-resistant...,2020
3,2020-07-31,"Zignale, M",The experienced space between mobility and Cov...,If it is true that lived space represents our ...,2020
4,2020-07-31,"Zhu, XC, Bodong; Avadhanam, Rukmini Manasa; Sh...",Reading and connecting: using social annotatio...,Purpose - The COM-19 pandemic has forced many ...,2020


In [8]:
df.iloc[0]['Abstract']

"Total out-of-pocket spending decreased over the same period; however, although the authors captured direct out-of-pocket spending on medical expenses, they did not include non-medical costs including loss of income, transport, and indirect economic costs due to tuberculosis (many of which are now being collected through WHO patient cost surveys) in their analysis. The authors' findings show that three countries with strong private sectors—Democratic Republic of the Congo, Nigeria, and Pakistan—have out-of-pocket medical expenses as the primary source of tuberculosis spending. [...]trends over time and across countries can be used to monitor fluctuations in total tuberculosis spending and assess needs across regions."

# Part 1

## Test Regex

In [9]:
pattern = r'\bItaly|Italian\b'
re.findall(pattern, 'Italy ITALY ITALIAN Italian italy italian ItaLian iTalY', re.IGNORECASE)

['Italy',
 'ITALY',
 'ITALIAN',
 'Italian',
 'italy',
 'italian',
 'ItaLian',
 'iTalY']

In [10]:
matches_l = []
for index, row in df.iterrows():
    abstract = row['Abstract']
    if isinstance(abstract, str):
        matches = re.findall(pattern, abstract, re.IGNORECASE)
        match_count = len(matches)
        if match_count > 0:
            matches_l.append((matches, match_count, abstract))
top_matches = sorted(matches_l, key=lambda x: x[1], reverse=True)[:10]

In [11]:
for i in top_matches:
    print(i[0])
    print(i[1])
    print(i[2])
    

['Italy', 'Italy', 'Italy', 'Italian', 'Italy', 'Italy', 'Italy', 'Italy', 'Italy', 'Italian', 'Italy', 'Italy']
12
Italy is currently experiencing an epidemic of COVID-19 which emerged in the Lombardy region . During the interval between February 25-29, 2020, we identified 46 cases of COVID-19 reported in 21 countries in Europe, Africa, North America, and South America which were either in individuals with recent travel from Italy, or who had presumed infection by a traveler from Italy 2. In six cases, in four of the affected countries (Switzerland, France, Austria, Croatia), land travel was a likely route of introduction, or was documented to have been the route of introduction. We used air travel volume between Italian cities and cities in other countries as an index of connectedness, using data available from the International Air Transport Association (IATA) for February 2015 (2.61 million total departing international air passengers from Italy). We used the methods of Fraser et a

# Part 2
## Test tokenizer

In [12]:
from nltk.tokenize import wordpunct_tokenize
text = 'This is a Italy of the ITALY. ITALIAN Italian in the realm. That; right there is italy "italian" ItaLian iTalY.'
tokens = wordpunct_tokenize(text)
print(tokens)

['This', 'is', 'a', 'Italy', 'of', 'the', 'ITALY', '.', 'ITALIAN', 'Italian', 'in', 'the', 'realm', '.', 'That', ';', 'right', 'there', 'is', 'italy', '"', 'italian', '"', 'ItaLian', 'iTalY', '.']


In [13]:
bigrams = Counter()
pattern = re.compile(r'\b(Italy|Italian)\b', re.IGNORECASE)
for index, row in df.iterrows():
    abstract = row['Abstract']
    if isinstance(abstract, str):
        tokens = wordpunct_tokenize(abstract)
        bigrams_list = list(ngrams(tokens, 2))
        re_search = [bigram for bigram in bigrams_list if pattern.search(bigram[0])]
        bigrams.update(re_search)

In [14]:
common_bigrams = bigrams.most_common(20)
for i in common_bigrams:
    print(str(i[0])+"\t"+ str(i[1]))

('Italy', ',')	703
('Italy', '.')	282
('Italy', 'and')	274
('Italy', 'is')	94
('Italy', '(')	79
('Italian', 'regions')	77
('Italy', 'has')	69
('Italian', 'Society')	57
('Italy', 'was')	56
('Italy', ')')	54
('Italian', 'population')	34
('Italian', 'government')	33
('Italian', 'National')	28
('Italian', 'Ministry')	28
('Italy', '),')	27
('Italy', 'in')	26
('Italian', 'Government')	23
('Italian', 'COVID')	19
('Italy', 'on')	18
('Italy', 'with')	18


# Part 3

## Test tokenizer

In [15]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/codespace/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [16]:
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))

In [17]:
text = 'This is a Italy of the ITALY. ITALIAN Italian in the realm. That; right there is italy "italian" ItaLian iTalY.'
tokens = wordpunct_tokenize(text)
cleaned_tokens = [token for token in tokens if token.isalpha() and token.lower() not in stop_words]
cleaned_tokens

['Italy',
 'ITALY',
 'ITALIAN',
 'Italian',
 'realm',
 'right',
 'italy',
 'italian',
 'ItaLian',
 'iTalY']

In [18]:
bigrams = Counter()
pattern = re.compile(r'\b(Italy|Italian)\b', re.IGNORECASE)
for index, row in df.iterrows():
    abstract = row['Abstract']
    if isinstance(abstract, str):
        tokens = wordpunct_tokenize(abstract)
        cleaned_tokens = [token for token in tokens if token.isalpha() and token.lower() not in stop_words]
        bigrams_list = list(ngrams(cleaned_tokens, 2))
        re_search = [bigram for bigram in bigrams_list if pattern.search(bigram[0])]
        bigrams.update(re_search)

In [19]:
common_bigrams = bigrams.most_common(20)
common_bigrams

[(('Italy', 'Spain'), 187),
 (('Italian', 'regions'), 77),
 (('Italy', 'one'), 57),
 (('Italy', 'Iran'), 57),
 (('Italian', 'Society'), 57),
 (('Italy', 'first'), 52),
 (('Italian', 'population'), 34),
 (('Italy', 'Germany'), 33),
 (('Italy', 'France'), 33),
 (('Italian', 'government'), 33),
 (('Italy', 'March'), 32),
 (('Italy', 'China'), 29),
 (('Italian', 'National'), 28),
 (('Italian', 'Ministry'), 28),
 (('Italy', 'South'), 27),
 (('Italy', 'February'), 26),
 (('Italy', 'USA'), 24),
 (('Italian', 'Government'), 23),
 (('Italy', 'United'), 20),
 (('Italian', 'COVID'), 19)]